
# Reconocimiento Facial con Isomap y Perceptrón Multicapa (MLP)

Este notebook implementa un pipeline completo de reconocimiento facial usando reducción de dimensionalidad con Isomap, clasificación con redes neuronales (MLP) y optimización de hiperparámetros con GridSearchCV. Se trabaja con imágenes originales y aumentadas organizadas por sujeto en carpetas individuales.


In [ ]:

from google.colab import drive
drive.mount('/content/drive')


In [ ]:

import os

base_path = '/content/drive/MyDrive/proyecto_isomap'
data_path = os.path.join(base_path, 'data')
fotos_path = os.path.join(data_path, 'fotos')
aumentadas_path = os.path.join(data_path, 'aumentadas')
processed_path = os.path.join(base_path, 'processed')
models_path = os.path.join(base_path, 'models')
results_path = os.path.join(base_path, 'results')

for path in [processed_path, models_path, results_path]:
    os.makedirs(path, exist_ok=True)


In [ ]:
!pip install joblib tqdm

In [ ]:

import numpy as np
from PIL import Image
from tqdm import tqdm

img_size = (64, 64)

def load_images_from_folder(folder):
    data = []
    labels = []
    label_names = sorted(os.listdir(folder))
    for label in label_names:
        label_path = os.path.join(folder, label)
        if not os.path.isdir(label_path):
            continue
        for file in os.listdir(label_path):
            try:
                img_path = os.path.join(label_path, file)
                img = Image.open(img_path).convert("L").resize(img_size)
                data.append(np.asarray(img).flatten())
                labels.append(label)
            except Exception as e:
                print(f"Error con {img_path}: {e}")
    return np.array(data), np.array(labels)


In [ ]:

import joblib

fotos_file = os.path.join(processed_path, 'X_fotos.joblib')
aug_file = os.path.join(processed_path, 'X_aug.joblib')

if os.path.exists(fotos_file) and os.path.exists(aug_file):
    X_fotos, y_fotos = joblib.load(fotos_file)
    X_aug, y_aug = joblib.load(aug_file)
else:
    X_fotos, y_fotos = load_images_from_folder(fotos_path)
    X_aug, y_aug = load_images_from_folder(aumentadas_path)
    joblib.dump((X_fotos, y_fotos), fotos_file)
    joblib.dump((X_aug, y_aug), aug_file)


In [ ]:

from sklearn.model_selection import train_test_split

X_fotos_train, X_test, y_fotos_train, y_test = train_test_split(
    X_fotos, y_fotos, test_size=0.3, stratify=y_fotos, random_state=42
)

X_train = np.concatenate([X_fotos_train, X_aug], axis=0)
y_train = np.concatenate([y_fotos_train, y_aug], axis=0)


In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.manifold import Isomap
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([
    ('iso', Isomap()),
    ('scaler', StandardScaler()),
    ('mlp', MLPClassifier(max_iter=500, random_state=42))
])

param_grid = {
    'iso__n_neighbors': [5, 10],
    'iso__n_components': [20, 40],
    'mlp__hidden_layer_sizes': [(100,), (50, 50)],
    'mlp__learning_rate_init': [0.001, 0.01],
    'mlp__activation': ['relu', 'tanh'],
    'mlp__alpha': [0.0001, 0.001],
}

grid = GridSearchCV(pipe, param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
grid.fit(X_train, y_train)


In [ ]:

from sklearn.metrics import classification_report

y_pred = grid.predict(X_test)
report = classification_report(y_test, y_pred)
print("Mejores hiperparámetros:", grid.best_params_)
print("\nReporte de clasificación:\n", report)

joblib.dump(grid.best_estimator_, os.path.join(models_path, 'mejor_modelo.joblib'))

with open(os.path.join(results_path, 'clasificacion.txt'), 'w') as f:
    f.write("Mejores hiperparámetros:\n")
    f.write(str(grid.best_params_))
    f.write("\n\nReporte de clasificación:\n")
    f.write(report)
